<a href="https://colab.research.google.com/github/uhyozzy/Project/blob/master/%EB%8B%A4%EB%B3%80%EC%9D%B8%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80%EB%AA%A8%EB%8D%B8_%EB%B0%8F_%EC%98%88%EC%B8%A1(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **■ 지역환경과 하수관로 비율 선형회귀 학습 후 예측**

# [1] TensorFlow를 이용한 **다변인 선형회귀 모델**

참고1 : https://www.youtube.com/watch?v=9Ur7bbZtoTc 혹은 https://ndb796.tistory.com/126

참고2 : https://youtu.be/ve6gtpZV83E

- 다변인 선형회귀는 모델에 영향을 미치는 변인이 여러 개 일 때 사용하는 모델임. 현재 우리의 데이터에서는 변인이 '강수량', '지형', '지역환경이므로 이 모든 변인이 '하수관로비율'에 영향을 미친다고 감안해야 함.
- 학습률을 0.000005로 설정하여 되도록 짧은 시간에 정확한 결과가 나오도록 구현

**Base**

In [3]:
import tensorflow as tf
import numpy as np
from pandas.io.parsers import read_csv

In [4]:
# 파일을 쉼표로 구분하여 읽어 데이터를 DataFrame 형태로 불러옴
data_origin = read_csv('/content/지역환경,하수관로.csv', sep=',')

In [5]:
# 결측치 확인
missing_values = data_origin.isnull().sum()
print(missing_values)

불투수면(%)               0
녹지 면적율(%)             0
하천 면적율(%)             0
복개하천 개수(개)            0
맨홀 개수(개)              0
빗물받이 개수(개)            0
빗물 펌프 개수(개)           0
[MAX] 하수관로 비율(%)    151
dtype: int64


In [6]:
# 결측치가 있는 행 제거한 후 확인
data = data_origin.dropna()
missing_values2 = data.isnull().sum()
print(missing_values2)

불투수면(%)             0
녹지 면적율(%)           0
하천 면적율(%)           0
복개하천 개수(개)          0
맨홀 개수(개)            0
빗물받이 개수(개)          0
빗물 펌프 개수(개)         0
[MAX] 하수관로 비율(%)    0
dtype: int64


In [7]:
# DataFrame을 NumPy 배열로 변환하고 데이터 타입을 float32로 지정
xy = np.array(data, dtype=np.float32)

In [8]:
# 7개의 변인을 입력을 받음
x_data = xy[:, 0:7]

In [9]:
# 하수관로(MAX) 값을 입력 받음
# 출력 데이터로 사용할 마지막 열의 데이터를 추출
y_data = xy[:, [-1]]

In [10]:
# 입력 데이터를 위한 TensorFlow placeholder를 생성합니다.
# 데이터 형태는 [None, 7]과 [None, 1]인데, None은 임의의 행 개수를 의미 -> X는 2개의 변인, y는 1개의 변인
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 7], name="X")
Y = tf.placeholder(tf.float32, shape=[None, 1], name="Y")

In [11]:
# 가중치 변수 W를 생성. 초기값은 정규 분포를 따르는 랜덤한 값임
W = tf.Variable(tf.random_normal([7, 1]), name="weight")

In [12]:
# 편향 변수 b를 생성
b = tf.Variable(tf.random_normal([1]), name="bias")

In [13]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [14]:
# 비용 함수를 설정합니다.(평균 제곱 오차를 계산 )
cost = tf.reduce_mean(tf.square(hypothesis - Y))

In [15]:
# 최적화 함수를 설정

# 경사 하강법 최적화기를 생성하고 학습률을 설정
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005)
# 비용 함수를 최소화하기 위한 훈련 오퍼레이션을 생성
train = optimizer.minimize(cost)

In [16]:
# 세션을 생성
sess = tf.Session()

In [17]:
# 글로벌 변수를 초기화
sess.run(tf.global_variables_initializer())

**학습 수행**
- ※주의※ Nan값 있으면 학습이 제대로 수행안되니 결측치 필수로 제거 후 학습 진행

In [18]:
# 학습을 수행합니다.

for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})
    if step % 500 == 0:
        print("#", step, " 손실 비용: ", cost_)
        print("- 하수관로비율(MAX): ", hypo_[0])

# 100001번 반복하는 학습 루프
# 학습을 수행하면서 비용, 가설 값, 훈련을 실행하여 비용 최소화를 진행
# 매 500번째 스텝마다 학습 상황에 대한 내용을 출력

# 0  손실 비용:  9.328221
- 하수관로비율(MAX):  [1.131656]
# 500  손실 비용:  9.210311
- 하수관로비율(MAX):  [1.1126232]
# 1000  손실 비용:  9.094597
- 하수관로비율(MAX):  [1.0938623]
# 1500  손실 비용:  8.980982
- 하수관로비율(MAX):  [1.0753268]
# 2000  손실 비용:  8.869413
- 하수관로비율(MAX):  [1.0570151]
# 2500  손실 비용:  8.759796
- 하수관로비율(MAX):  [1.038968]
# 3000  손실 비용:  8.652227
- 하수관로비율(MAX):  [1.021163]
# 3500  손실 비용:  8.546638
- 하수관로비율(MAX):  [1.0035836]
# 4000  손실 비용:  8.442915
- 하수관로비율(MAX):  [0.9862137]
# 4500  손실 비용:  8.340968
- 하수관로비율(MAX):  [0.9691025]
# 5000  손실 비용:  8.240897
- 하수관로비율(MAX):  [0.9522142]
# 5500  손실 비용:  8.14267
- 하수관로비율(MAX):  [0.9355453]
# 6000  손실 비용:  8.046147
- 하수관로비율(MAX):  [0.9190747]
# 6500  손실 비용:  7.9512553
- 하수관로비율(MAX):  [0.9028396]
# 7000  손실 비용:  7.858143
- 하수관로비율(MAX):  [0.8868414]
# 7500  손실 비용:  7.7666674
- 하수관로비율(MAX):  [0.8710405]
# 8000  손실 비용:  7.6767435
- 하수관로비율(MAX):  [0.85542524]
# 8500  손실 비용:  7.5884266
- 하수관로비율(MAX):  [0.84001553]
# 9000  손실 비용:  7.5016785
- 하수관로비율(MAX):  [0.824

**학습 모델 저장**

In [19]:
saver = tf.train.Saver()
save_path = saver.save(sess, "./saved.cpkt")
print('학습된 모델을 저장했습니다.')

# 모델 저장을 위한 Saver 객체를 생성
# 학습된 모델을 저장
# 저장 완료 메시지를 출력

학습된 모델을 저장했습니다.


# [2] 다변인 선형회귀를 활용한 **하수관로 비율 예측**

**Base**

In [20]:
import tensorflow as tf
import numpy as np

In [21]:
# 플레이스 홀더를 설정.
# 입력 데이터와 출력 데이터를 위한 TensorFlow placeholder를 생성.
# 입력 데이터는 [None, 7] 모양이고 출력 데이터는 [None, 1] 모양임.

import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

X = tf.placeholder(tf.float32, shape=[None, 7])
Y = tf.placeholder(tf.float32, shape=[None, 1])

In [22]:
#가중치와 편향 변수를 생성. 가중치의 모양은 [7, 1]이고 편향의 모양은 [1]

W = tf.Variable(tf.random_normal([7, 1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

In [23]:
# 선형 회귀 모델의 가설을 설정
hypothesis = tf.matmul(X, W) + b

In [24]:
# TensorFlow 그래프와 세션 설정
tf.compat.v1.disable_eager_execution()
sess = tf.compat.v1.Session()

# 학습 모델 불러오기
save_path = "./saved.cpkt"
saver = tf.compat.v1.train.import_meta_graph(save_path + ".meta")
saver.restore(sess, save_path)

# 그래프로부터 텐서 가져오기
graph = tf.compat.v1.get_default_graph()
X = graph.get_tensor_by_name("X:0")  # Placeholder 이름 변경
hypothesis = graph.get_tensor_by_name("add:0")  # hypothesis 텐서 이름 변경


In [29]:
# 사용자로부터 강수량을 입력받습니다.

impervious = float(input('불투수면 (%) : '))
greenery = float(input('녹지 면적율(%) : '))
river = float(input('하천 면적율(%): '))
covered_river = float(input('복개하천 개수(개) : '))
manhole = float(input('맨홀 개수(개)) : '))
rain_gutter = float(input('빗물 1받이 개수(개) : '))
pump = float(input('빗물 펌프 개수(개) : '))

불투수면 (%) : 40
녹지 면적율(%) : 9
하천 면적율(%): 10
복개하천 개수(개) : 1
맨홀 개수(개)) : 9000
빗물 받이 개수(개) : 16000
빗물 펌프 개수(개) : 1


In [30]:
# 입력 데이터를 준비
data = np.array([[impervious,greenery,river,covered_river,
                  manhole,rain_gutter,pump]], dtype=np.float32)

feed_dict = {X: data}
prediction = sess.run(hypothesis, feed_dict=feed_dict)

print("예측된 하수관로비율(MAX):", prediction[0][0])

예측된 하수관로비율(MAX): 2637.882
